너는 반도체 공정/계측 데이터 기반 품질 판정 모델을 만드는 데이터 사이언티스트다.
아래 주어진 피처(계측/공정/장비 상태)를 사용해 “결함 여부(불량/정상)” 이진 분류 모델을 구축한다.
나는 재현 가능한 파이썬 코드(주피터 노트북 형태 가능)로 EDA→전처리→모델링→평가→해석→운영까지 한 번에 원한다.

[데이터]
| 컬럼명 | 데이터 타입 (예시) | 설명 | EDA 분석 포인트 |
| --- | --- | --- | --- |
| **Process_ID** | String | 공정 작업 고유 ID | 중복 데이터 확인 및 작업 단위 식별 |
| **Timestamp** | DateTime | 데이터 기록 시각 | 시간 흐름에 따른 공정 안정성 및 트렌드 분석 |
| **Tool_Type** | Categorical | 공정 설비 종류 (Lithography 등) | 설비별 불량률 차이 분석 |
| **Wafer_ID** | String | 웨이퍼 고유 식별 번호 | 특정 웨이퍼 로트(Lot)의 문제 여부 확인 |
| **Chamber_Temperature** | Float | 챔버 내부 온도 | 온도와 결함 간의 상관관계(상하한 임계치) |
| **Gas_Flow_Rate** | Float | 주입 가스 유량 | 가스 공급 안정성이 품질에 미치는 영향 |
| **RF_Power** | Float | 무선 주파수 전력 강도 | 에너지 세기와 식각/증착 품질의 관계 |
| **Etch_Depth** | Float | 식각(Etching) 깊이 | 목표 깊이 대비 편차 분석 |
| **Rotation_Speed** | Float | 장비/웨이퍼 회전 속도(RPM) | 회전 균일도와 코팅/식각 균일도 관계 |
| **Vacuum_Pressure** | Float | 챔버 내 진공 압력 | 압력 변화에 따른 파티클 발생 및 공정 오류 |
| **Stage_Alignment_Error** | Float | 스테이지 정렬 오차 | 정밀도 오류와 결함(Defect) 간의 상관성 |
| **Vibration_Level** | Float | 설비 미세 진동 수치 | 진동이 공정 정밀도에 미치는 영향 |
| **UV_Exposure_Intensity** | Float | UV 노광 강도 | 노광량 부족/과다에 따른 패턴 결함 분석 |
| **Particle_Count** | Integer | 미세 먼지(파티클) 개수 | 환경 오염도와 결함 여부의 직접적 연관성 |
| **Defect** | Binary (0, 1) | 결함 발생 여부 (Target) | 분류 모델의 종속 변수 (0: 정상, 1: 불량) |
| **Join_Status** | Categorical | 최종 공정 통과 상태 | 최종 판정 결과 (Joining: 합격, Non-Joining: 불량) |

- 타깃(라벨) 컬럼명: defect_flag (또는 내가 제공하는 y 컬럼)
  * 이진 분류: 1=결함/불량, 0=정상

- 추가로 있을 수 있는 컬럼(있으면 자동 인식):
  lot_id / wafer_id / die_id / timestamp / equipment_id / recipe_id
  (없어도 진행하되, 누수 위험을 경고하고 대안을 제시)

[핵심 주의사항: 누수/운영정합성]
- Etch_Depth는 “공정 결과 계측” 성격이므로,
  운영 시점에 결함 판정 전에 얻을 수 있는 값인지가 중요하다.
  따라서 아래 2가지 시나리오를 모두 수행하고 성능/해석을 비교해라:
  (A) Etch_Depth 포함 모델 (결과 계측까지 포함한 사후 판정)
  (B) Etch_Depth 제외 모델 (공정 조건+장비 상태만으로 사전/중간 판정)
- 그룹/시간 누수가 가능하다면 GroupKFold(우선 lot_id, 없으면 wafer_id) 또는 time-aware split을 적용한다.
  ID가 전혀 없으면, 랜덤 분할은 위험하다는 경고를 출력하고,
  최소한 StratifiedKFold와 “데이터 수집 단위가 섞였을 가능성”을 리포트에 적어라.

* **Target 변수**: `Defect` 또는 `Join_Status`를 예측 목표로 삼을 수 있습니다. 두 변수가 거의 동일한 정보를 담고 있는지 확인이 필요합니다.
* **센서 데이터 특성**: 온도, 압력, 전력 등은 서로 밀접한 관계가 있을 가능성이 높으므로 다중공선성(Multicollinearity) 확인이 유용합니다.
* **공정별 차이**: `Tool_Type`에 따라 주요하게 작용하는 센서(예: Lithography의 경우 UV Intensity)가 다를 수 있으니 그룹화 분석을 추천합니다.

[산출물 요구사항]
1) 먼저 “필수 확인 질문 5개”만 제시해라. 단, 내가 답을 안 해도 디폴트로 즉시 진행해라.
2) EDA:
   - 각 피처의 분포, 이상치 후보(예: IQR 기준), 결측 비율
   - 타깃 비율(불량률)
   - 상관/다중공선성 힌트(heatmap 또는 요약)
   - Particle_Count, Vibration_Level, Stage_Alignment_Error 같은 품질 지표의 tail 분석
3) 전처리:
   - processID, timestamp 는 무의미한 데이터이므로 제거
   - 결측치: SimpleImputer(median) 기본 + IterativeImputer 대안 비교(옵션)
   - 스케일링: RobustScaler(이상치 강건)
   - 이상치 대응: winsorization(상하 1~2%) 또는 IsolationForest(선택 기준 설명)
4) 모델링:
   - AutoML 로 생성
   - 최적 모델 산출
5) 검증/평가:
   - 주요 지표: PR-AUC(우선), ROC-AUC(참고), F1, Recall@Precision>=0.90
   - PR curve, confusion matrix
   - 최적 threshold를 “Precision 제약(>=0.90)” 하에서 Recall 최대화로 찾는 로직 구현
   - Calibration curve(가능하면) + Platt/Isotonic calibration 옵션
6) 해석:
   - Permutation importance(기본)
   - (가능하면) SHAP 옵션 제시
   - 물리적으로 말이 되는지 sanity check 코멘트:
     예: Vacuum_Pressure↑가 etch 안정성/particle에 미치는 영향, Alignment/Vibration이 결함 증가로 이어지는 가설 등
7) 운영:
   - sklearn Pipeline으로 학습/추론 일체화
   - 모델 저장(joblib)
   - 입력 컬럼 스키마 검증(누락/추가 컬럼 탐지)
   - 드리프트 모니터링 제안(PSI/KS, Particle_Count 등 핵심 피처)

[출력 형식]
- 한국어로 설명, 코드는 파이썬 코드블록으로 섹션별 제공
- 사용자가 바꿔 끼울 수 있게 상단에 설정 변수(CONFIG)로 컬럼명/타깃/그룹/시간을 모아라.
- 외부 데이터 사용 금지. 주어진 CSV만 사용.
- 데이터/컬럼이 없으면 가정 A/B로 분기해라.

이제 위 요구사항을 만족하는 “실행 가능한 전체 코드 + 해석 가이드”를 작성해라.